One of my mentees did something really cool
https://colab.research.google.com/drive/12MTYkd6eZFbJ7tcuUxhNjStnwLdFunpL#scrollTo=6c219485

Here are some resources to look into
https://www.geeksforgeeks.org/python/pyyaml-fullloader-python/

https://medium.com/python-in-plain-english/using-yaml-file-as-a-small-db-in-python-68ebc61dd490
https://medium.com/@ThinkingLoop/10-pandas-data-cleaning-moves-that-saved-my-career-2fe4e8a50f18



These are the various steps we will take

1. Removing columns that are missing significantly data points  - creating a null report 📁 
2. Removing duplicates✅
2. Removing betting odds columns - this is like stealing✅
3. Confirm whether data columns types are correctly mapped
4. Canonicalize text (and stop death-by-typo) - cleaning text data with regular expressions, fixing cases
4. Normalize date formatsClean and create new date columns - year , month 

# Normalize date formats
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])

5. One-hot and categorical the smart way - High-cardinality strings are memory hogs. Use category and selective encoding.
6. Automate validation checks using tools like Great Expectations.
7. Passing data to Neon Db - Using yaml files

import yaml
with open("configs/yaml" , "r") as f:
    data = yaml.safe_load(f)
username = data["production"]["username"]







Mindset shifts that changed everything
1. **Be explicit**. If a rule isn’t encoded in code, it’s a rumor.
2. **Prefer immutability**. Reassign results; avoid inplace=True for clarity and chaining.
3. **Choose honesty over prettiness**. A lower KPI with correct math beats a rosy one built on accidental exclusions.
4. **Automate small checks**. Five assertions can protect a million rows.

In [1]:
import pandas as pd
from pathlib import Path

# ===== Defining the data directories ====#
PROJECT_ROOT = Path().absolute().parent.parent
DATA_DIR = PROJECT_ROOT / "datasets" 
COMMON_DATA_DIR = DATA_DIR / "common_data"
INGESTED_DIR = DATA_DIR / "ingested_data"
ingested_csv_file = INGESTED_DIR / "enhanced_dataset.csv"

In [2]:
print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Common data directory: {COMMON_DATA_DIR}")
print(f"📁 Ingested data directory: {INGESTED_DIR}")

📁 Project root: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores
📁 Data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets
📁 Common data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets\common_data
📁 Ingested data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets\ingested_data


In [3]:
import pandas as pd
data = pd.read_csv(ingested_csv_file)
data.head()

C:\Users\juliu\AppData\Local\Temp\ipykernel_852\3313027951.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(ingested_csv_file)


,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,SJA,GBH,GBD,GBA,BSH,BSD,BSA,SBH,SBD,SBA
0,15/08/2025,20:00,Liverpool,Bournemouth,4.0,2.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/08/2025,12:30,Aston Villa,Newcastle,0.0,0.0,D,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16/08/2025,15:00,Brighton,Fulham,1.0,1.0,D,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16/08/2025,15:00,Sunderland,West Ham,3.0,0.0,H,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16/08/2025,15:00,Tottenham,Burnley,3.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.shape

(7081, 193)

In [5]:
data.columns

Index(['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR',
       ...
       'SJA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'SBH', 'SBD', 'SBA'],
      dtype='object', length=193)

**Initial Deletion of columns - columns that are mostly empty**

In [6]:
print(f"The number of columns are ",data.shape[1], " and the number of the rows are",data.shape[0] )

The number of columns are  193  and the number of the rows are 7081


In [7]:
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,percent],axis=1, keys=['Total','Percent'])
missing_data.head(20)

,Total,Percent
LBCD,6861,0.968931
CLCH,6861,0.968931
CLCA,6861,0.968931
LBCA,6861,0.968931
CLCD,6861,0.968931
LBCH,6861,0.968931
CLH,6859,0.968648
CLD,6859,0.968648
CLA,6859,0.968648
BMGMH,6841,0.966106


In [8]:
new_data = data.drop(missing_data[missing_data['Percent'] > 0.10].index, axis=1)
new_data.isnull().sum()
new_data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,BWD,BWA,season_id,competition_name,WHH,WHD,WHA,VCH,VCD,VCA
0,15/08/2025,Liverpool,Bournemouth,4.0,2.0,H,1.0,0.0,H,A Taylor,...,5.5,7.50,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
1,16/08/2025,Aston Villa,Newcastle,0.0,0.0,D,0.0,0.0,D,C Pawson,...,3.6,2.87,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
2,16/08/2025,Brighton,Fulham,1.0,1.0,D,0.0,0.0,D,S Barrott,...,3.6,3.90,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
3,16/08/2025,Sunderland,West Ham,3.0,0.0,H,0.0,0.0,D,R Jones,...,3.4,2.25,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
4,16/08/2025,Tottenham,Burnley,3.0,0.0,H,1.0,0.0,H,M Oliver,...,4.8,8.00,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
new_data.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA',
       'season_id', 'competition_name', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD',
       'VCA'],
      dtype='object')

In [10]:
print(f"The number of columns are ",new_data.shape[1], " and the number of the rows are",new_data.shape[0] )

The number of columns are  36  and the number of the rows are 7081


In [11]:
total = new_data.isnull().sum().sort_values(ascending=False)
percent = (new_data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
total_rows = new_data.shape[0]


missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
VCD,621.0,0.087699
VCA,621.0,0.087699
VCH,621.0,0.087699
WHD,439.0,0.061997
WHH,439.0,0.061997
WHA,439.0,0.061997
BWA,145.0,0.020477
BWD,145.0,0.020477
BWH,145.0,0.020477
AwayTeam,1.0,0.000141


In [12]:
print(f"The number of columns are ",new_data.shape[1], " and the number of the rows are",new_data.shape[0] )

The number of columns are  36  and the number of the rows are 7081


In [13]:
new_data.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,BWD,BWA,season_id,competition_name,WHH,WHD,WHA,VCH,VCD,VCA
0,15/08/2025,Liverpool,Bournemouth,4.0,2.0,H,1.0,0.0,H,A Taylor,...,5.50,7.50,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
1,16/08/2025,Aston Villa,Newcastle,0.0,0.0,D,0.0,0.0,D,C Pawson,...,3.60,2.87,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
2,16/08/2025,Brighton,Fulham,1.0,1.0,D,0.0,0.0,D,S Barrott,...,3.60,3.90,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
3,16/08/2025,Sunderland,West Ham,3.0,0.0,H,0.0,0.0,D,R Jones,...,3.40,2.25,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
4,16/08/2025,Tottenham,Burnley,3.0,0.0,H,1.0,0.0,H,M Oliver,...,4.80,8.00,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
5,16/08/2025,Wolves,Man City,0.0,4.0,A,0.0,2.0,A,J Gillett,...,4.60,1.46,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
6,17/08/2025,Chelsea,Crystal Palace,0.0,0.0,D,0.0,0.0,D,D England,...,4.20,5.25,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
7,17/08/2025,Nott'm Forest,Brentford,3.0,1.0,H,3.0,0.0,H,P Bankes,...,3.40,3.40,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
8,17/08/2025,Man United,Arsenal,0.0,1.0,A,0.0,1.0,A,S Hooper,...,3.50,2.10,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
9,18/08/2025,Leeds,Everton,1.0,0.0,H,0.0,0.0,D,C Kavanagh,...,3.25,2.95,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
new_data.sample(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,BWD,BWA,season_id,competition_name,WHH,WHD,WHA,VCH,VCD,VCA
1609,13/02/2022,Burnley,Liverpool,0.0,1.0,A,0.0,1.0,A,M Atkinson,...,5.50,1.30,2021-2022,EPL,11.00,5.25,1.29,11.50,5.50,1.25
478,03/02/2025,Chelsea,West Ham,2.0,1.0,H,0.0,1.0,A,S Attwell,...,5.75,7.25,2024-2025,EPL,1.36,5.00,7.00,NaN,NaN,NaN
2204,28/09/2019,Crystal Palace,Norwich,2.0,0.0,H,1.0,0.0,H,J Moss,...,3.60,3.80,2019-2020,EPL,1.91,3.70,3.90,1.90,3.80,4.00
6116,20/12/09,Wolves,Burnley,2.0,0.0,H,1.0,0.0,H,M Atkinson,...,3.25,3.35,2009-2010,EPL,2.00,3.40,3.75,2.10,3.25,3.75
4156,22/11/14,Stoke,Burnley,1.0,2.0,A,1.0,2.0,A,M Atkinson,...,3.60,4.75,2014-2015,EPL,1.73,3.10,6.00,1.73,3.75,5.50
6728,25/08/07,Arsenal,Man City,1.0,0.0,H,0.0,0.0,D,C Foy,...,3.80,5.20,2007-2008,EPL,NaN,NaN,NaN,1.65,3.30,5.50
6910,12/01/08,Arsenal,Birmingham,1.0,1.0,D,1.0,0.0,H,P Dowd,...,6.50,11.00,2007-2008,EPL,1.17,5.50,12.00,1.17,6.00,17.00
6316,09/05/10,Everton,Portsmouth,1.0,0.0,H,0.0,0.0,D,P Walton,...,5.50,10.00,2009-2010,EPL,1.29,5.50,10.00,1.22,6.00,12.00
4126,26/10/14,Burnley,Everton,1.0,3.0,A,1.0,2.0,A,A Marriner,...,3.60,1.85,2014-2015,EPL,4.33,3.60,1.75,4.60,3.75,1.85
2975,14/10/2017,Tottenham,Bournemouth,1.0,0.0,H,0.0,0.0,D,R Madley,...,7.25,14.00,2017-2018,EPL,1.20,7.50,13.00,1.22,7.50,13.00


In [15]:
new_data.sample(frac=0.002)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,BWD,BWA,season_id,competition_name,WHH,WHD,WHA,VCH,VCD,VCA
4305,28/02/15,West Ham,Crystal Palace,1.0,3.0,A,0.0,1.0,A,M Dean,...,3.40,3.50,2014-2015,EPL,2.15,3.10,3.60,2.15,3.40,3.80
5118,13/04/13,Aston Villa,Fulham,1.0,1.0,D,0.0,0.0,D,P Dowd,...,3.20,3.25,2012-2013,EPL,2.30,3.10,3.30,2.38,3.40,3.20
1622,20/02/2022,Leeds,Man United,2.0,4.0,A,0.0,2.0,A,P Tierney,...,3.80,1.87,2021-2022,EPL,3.90,3.75,1.85,3.90,3.70,1.85
6981,08/03/08,Liverpool,Newcastle,3.0,0.0,H,2.0,0.0,H,P Walton,...,4.00,7.50,2007-2008,EPL,1.36,3.75,8.00,1.40,4.35,10.00
4230,01/01/15,Aston Villa,Crystal Palace,0.0,0.0,D,0.0,0.0,D,R Madley,...,3.20,2.90,2014-2015,EPL,2.40,3.10,3.10,2.40,3.20,3.40
4159,24/11/14,Aston Villa,Southampton,1.0,1.0,D,1.0,0.0,H,P Dowd,...,3.60,1.78,2014-2015,EPL,5.00,3.20,1.80,5.40,3.75,1.75
4878,21/10/12,QPR,Everton,1.0,1.0,D,1.0,1.0,D,J Moss,...,3.40,2.25,2012-2013,EPL,3.20,3.20,2.30,3.40,3.50,2.20
4201,20/12/14,Hull,Swansea,0.0,1.0,A,0.0,1.0,A,M Clattenburg,...,3.25,2.60,2014-2015,EPL,2.80,3.10,2.60,3.10,3.25,2.55
237,01/02/2026,Nott'm Forest,Crystal Palace,1.0,1.0,D,1.0,1.0,D,M Salisbury,...,3.50,3.70,2025-2026,EPL,NaN,NaN,NaN,NaN,NaN,NaN
5113,07/04/13,Newcastle,Fulham,1.0,0.0,H,0.0,0.0,D,K Friend,...,3.25,3.90,2012-2013,EPL,2.05,3.40,3.60,2.15,3.60,3.50


In [16]:
print(new_data.sample(10))


            Date        HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
3535    25/02/17       West Brom  Bournemouth   2.0   1.0   H   2.0   1.0   H   
5313    03/12/11       Newcastle      Chelsea   0.0   3.0   A   0.0   1.0   A   
858   17/02/2024       Brentford    Liverpool   1.0   4.0   A   0.0   1.0   A   
760   05/12/2023          Wolves      Burnley   1.0   0.0   H   1.0   0.0   H   
2976  14/10/2017         Watford      Arsenal   2.0   1.0   H   0.0   1.0   A   
5731    18/12/10      Sunderland       Bolton   1.0   0.0   H   1.0   0.0   H   
4869    07/10/12       Tottenham  Aston Villa   2.0   0.0   H   0.0   0.0   D   
1041  30/08/2022          Fulham     Brighton   2.0   1.0   H   0.0   0.0   D   
2361  18/01/2020        Brighton  Aston Villa   1.0   1.0   D   1.0   0.0   H   
3701  12/09/2015  Crystal Palace     Man City   0.0   1.0   A   0.0   0.0   D   

            Referee  ...   BWD   BWA  season_id  competition_name   WHH   WHD  \
3535  M Clattenburg  ...  3

In [17]:
null_report = (
    new_data.isna()
      .mean()
      .rename("missing_rate")
      .to_frame()
      .assign(n_null=new_data.isna().sum())
      .sort_values("missing_rate", ascending=False)
)
null_report

,missing_rate,n_null
VCD,0.087699,621
VCA,0.087699,621
VCH,0.087699,621
WHD,0.061997,439
WHH,0.061997,439
WHA,0.061997,439
BWA,0.020477,145
BWD,0.020477,145
BWH,0.020477,145
AwayTeam,0.000141,1


### Duplicate rows removal


In [18]:
#identify duplicate rows
duplicateRows = new_data[new_data.duplicated()]

#view duplicate rows
duplicateRows

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,BWD,BWA,season_id,competition_name,WHH,WHD,WHA,VCH,VCD,VCA


### Incorrect data types

In [19]:
new_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7081 entries, 0 to 7080
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              7080 non-null   object 
 1   HomeTeam          7080 non-null   object 
 2   AwayTeam          7080 non-null   object 
 3   FTHG              7080 non-null   float64
 4   FTAG              7080 non-null   float64
 5   FTR               7080 non-null   object 
 6   HTHG              7080 non-null   float64
 7   HTAG              7080 non-null   float64
 8   HTR               7080 non-null   object 
 9   Referee           7080 non-null   object 
 10  HS                7080 non-null   float64
 11  AS                7080 non-null   float64
 12  HST               7080 non-null   float64
 13  AST               7080 non-null   float64
 14  HF                7080 non-null   float64
 15  AF                7080 non-null   float64
 16  HC                7080 non-null   float64


In [20]:
new_df = new_data.dropna()

print(new_df.to_string())

            Date          HomeTeam          AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR        Referee    HS    AS   HST   AST    HF    AF    HC    AC   HY   AY   HR   AR  B365H  B365D  B365A    BWH    BWD    BWA  season_id competition_name    WHH    WHD    WHA     VCH    VCD     VCA
620   11/08/2023           Burnley          Man City   0.0   3.0   A   0.0   2.0   A       C Pawson   6.0  17.0   1.0   8.0  11.0   8.0   6.0   5.0  0.0  0.0  1.0  0.0   8.00   5.50   1.33   8.75   5.25   1.34  2023-2024              EPL   8.00   5.00   1.25   9.500   5.25   1.330
621   12/08/2023           Arsenal     Nott'm Forest   2.0   1.0   H   2.0   0.0   H       M Oliver  15.0   6.0   7.0   2.0  12.0  12.0   8.0   3.0  2.0  2.0  0.0  0.0   1.18   7.00  15.00   1.17   7.50  15.50  2023-2024              EPL   1.12   6.50  12.00   1.140   7.50  17.000
622   12/08/2023       Bournemouth          West Ham   1.0   1.0   D   0.0   0.0   D       P Bankes  14.0  16.0   5.0   3.0   9.0  14.0  10.0   4.0  1.0  

In [21]:
new_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7081 entries, 0 to 7080
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              7080 non-null   object 
 1   HomeTeam          7080 non-null   object 
 2   AwayTeam          7080 non-null   object 
 3   FTHG              7080 non-null   float64
 4   FTAG              7080 non-null   float64
 5   FTR               7080 non-null   object 
 6   HTHG              7080 non-null   float64
 7   HTAG              7080 non-null   float64
 8   HTR               7080 non-null   object 
 9   Referee           7080 non-null   object 
 10  HS                7080 non-null   float64
 11  AS                7080 non-null   float64
 12  HST               7080 non-null   float64
 13  AST               7080 non-null   float64
 14  HF                7080 non-null   float64
 15  AF                7080 non-null   float64
 16  HC                7080 non-null   float64


In [22]:
def inc(x):
    return x + 1


def test_answer():
    assert inc(4) == 5

test_answer()

In [23]:
#Standardize column names: lowercase, snake_case, strip spaces

#Borrowed from - Phoebe


#cohort3_DS_clean = cohort3_DS.copy()
#cohort3_DS_clean.columns = (
   # cohort3_DS_clean.columns
  #  .str.strip()
  #  .str.lower()
   # .str.replace(" ", "_")
    #.str.replace("?", "", regex=False)
#)
#cohort3_DS_clean.head()

# Data Cleaning (begins here)

##  1. Removing Betting Columns

In [24]:

import pandas as pd
from pathlib import Path

# ===== Defining the data directories ====#
PROJECT_ROOT = Path().absolute().parent.parent
DATA_DIR = PROJECT_ROOT / "datasets" 
COMMON_DATA_DIR = DATA_DIR / "common_data"
INGESTED_DIR = DATA_DIR / "ingested_data"

# == Ingested csv file == #
ingested_csv_file = INGESTED_DIR / "enhanced_dataset.csv"

# == Feature catalog csv == #
feature_catalog_csv_file = COMMON_DATA_DIR / "feature_catalog.csv"


print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Common data directory: {COMMON_DATA_DIR}")
print(f"📁 Ingested data directory: {INGESTED_DIR}")

📁 Project root: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores
📁 Data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets
📁 Common data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets\common_data
📁 Ingested data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets\ingested_data


In [25]:
# Enhanced dataset
enhanced_data = pd.read_csv(ingested_csv_file)
enhanced_data.sample(3)


C:\Users\juliu\AppData\Local\Temp\ipykernel_852\670297540.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  enhanced_data = pd.read_csv(ingested_csv_file)


,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,SJA,GBH,GBD,GBA,BSH,BSD,BSA,SBH,SBD,SBA
3390,19/11/16,NaN,Crystal Palace,Man City,1.0,2.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3328,18/09/16,NaN,Tottenham,Sunderland,1.0,0.0,H,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6026,19/10/09,NaN,Fulham,Hull,2.0,0.0,H,1.0,0.0,H,...,5.5,1.7,3.5,4.9,1.67,3.5,5.0,1.65,3.5,4.75


In [26]:
enhanced_data.dtypes

Date         object
Time         object
HomeTeam     object
AwayTeam     object
FTHG        float64
             ...   
BSD         float64
BSA         float64
SBH         float64
SBD         float64
SBA         float64
Length: 193, dtype: object

In [27]:
# === Removing betting odds features from the dataset === #
# Feature columns dataset
feature_catalog = pd.read_csv(feature_catalog_csv_file)
feature_catalog.sample(3)

,Feature_name,Feature_description,Betting_odd
69,PSD / PD,Pinnacle draw odds,True
25,AO,Away Team Offsides,False
28,HR,Home Team Red Cards,False


In [28]:
enhanced_data.shape

(7081, 193)

In [29]:
# === Using MultiIndex with isin() === #

filtered_feature_catalog = feature_catalog[feature_catalog["Betting_odd"]==False].copy()
filtered_feature_catalog.sample(2)



,Feature_name,Feature_description,Betting_odd
4,AwayTeam,Away Team,False
14,HST,Home Team Shots on Target,False


In [30]:

# 1. Get the column names to keep as a set for fast lookup
feature_names_to_keep = filtered_feature_catalog['Feature_name'].unique()
enhanced_data = enhanced_data.loc[:, enhanced_data.columns.isin(feature_names_to_keep)].copy()
enhanced_data

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,15/08/2025,20:00,Liverpool,Bournemouth,4.0,2.0,H,1.0,0.0,H,...,10.0,3.0,7.0,10.0,6.0,7.0,1.0,2.0,0.0,0.0
1,16/08/2025,12:30,Aston Villa,Newcastle,0.0,0.0,D,0.0,0.0,D,...,3.0,3.0,13.0,11.0,3.0,6.0,1.0,1.0,1.0,0.0
2,16/08/2025,15:00,Brighton,Fulham,1.0,1.0,D,0.0,0.0,D,...,4.0,2.0,16.0,15.0,4.0,3.0,3.0,3.0,0.0,0.0
3,16/08/2025,15:00,Sunderland,West Ham,3.0,0.0,H,0.0,0.0,D,...,5.0,4.0,8.0,10.0,5.0,7.0,0.0,1.0,0.0,0.0
4,16/08/2025,15:00,Tottenham,Burnley,3.0,0.0,H,1.0,0.0,H,...,6.0,4.0,14.0,8.0,6.0,5.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7076,11/05/08,NaN,Portsmouth,Fulham,0.0,1.0,A,0.0,0.0,D,...,10.0,4.0,13.0,11.0,5.0,6.0,0.0,0.0,0.0,0.0
7077,11/05/08,NaN,Sunderland,Arsenal,0.0,1.0,A,0.0,1.0,A,...,5.0,8.0,14.0,12.0,3.0,4.0,1.0,1.0,0.0,0.0
7078,11/05/08,NaN,Tottenham,Liverpool,0.0,2.0,A,0.0,0.0,D,...,3.0,12.0,11.0,9.0,8.0,4.0,1.0,1.0,0.0,0.0
7079,11/05/08,NaN,West Ham,Aston Villa,2.0,2.0,D,1.0,1.0,D,...,8.0,8.0,9.0,21.0,7.0,8.0,2.0,2.0,0.0,0.0


##  2. Removing columns missing a significant chunk of data  - creating null report

In [31]:
print(f"The number of columns are ",enhanced_data.shape[1], " and the number of the rows are",enhanced_data.shape[0] )

The number of columns are  23  and the number of the rows are 7081


The percent series is the result of a division, but when you concatenate it with the total series, the resulting 'Percent' column in missing_data might be stored as an object type (or sometimes a type that prevents direct numeric comparison)

total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,percent],axis=1, keys=['Total','Percent'])
missing_data.head(20)

In [32]:
total = enhanced_data.isnull().sum().sort_values(ascending=False)
missing_percent = (enhanced_data.isnull().sum() / len(enhanced_data)) * 100
missing_data = pd.concat([total,missing_percent],axis=1, keys=['Total','Percent'])
missing_data.head(20)

,Total,Percent
Time,4561,64.411806
Date,1,0.014122
HomeTeam,1,0.014122
AwayTeam,1,0.014122
FTHG,1,0.014122
FTAG,1,0.014122
FTR,1,0.014122
HTHG,1,0.014122
HTAG,1,0.014122
HTR,1,0.014122


In [33]:
#missing_percent = (enhanced_data.isnull().sum() / len(enhanced_data)) * 100

# 2. Identify the column names (the Series index) where the percentage is > 10
columns_to_drop = missing_percent[missing_percent > 10].index

# 3. Drop all identified columns in a single operation
enhanced_data_with_no_nulls = enhanced_data.drop(columns=columns_to_drop).copy()

# The .copy() is used here at the end to ensure the result is an independent DataFrame.
print(f"Dropped {len(columns_to_drop)} columns.")
print(f"Dropped {columns_to_drop} columns.")
print(enhanced_data_with_no_nulls.head())

Dropped 1 columns.
Dropped Index(['Time'], dtype='object') columns.
         Date     HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  15/08/2025    Liverpool  Bournemouth   4.0   2.0   H   1.0   0.0   H   
1  16/08/2025  Aston Villa    Newcastle   0.0   0.0   D   0.0   0.0   D   
2  16/08/2025     Brighton       Fulham   1.0   1.0   D   0.0   0.0   D   
3  16/08/2025   Sunderland     West Ham   3.0   0.0   H   0.0   0.0   D   
4  16/08/2025    Tottenham      Burnley   3.0   0.0   H   1.0   0.0   H   

     Referee  ...   HST  AST    HF    AF   HC   AC   HY   AY   HR   AR  
0   A Taylor  ...  10.0  3.0   7.0  10.0  6.0  7.0  1.0  2.0  0.0  0.0  
1   C Pawson  ...   3.0  3.0  13.0  11.0  3.0  6.0  1.0  1.0  1.0  0.0  
2  S Barrott  ...   4.0  2.0  16.0  15.0  4.0  3.0  3.0  3.0  0.0  0.0  
3    R Jones  ...   5.0  4.0   8.0  10.0  5.0  7.0  0.0  1.0  0.0  0.0  
4   M Oliver  ...   6.0  4.0  14.0   8.0  6.0  5.0  0.0  0.0  0.0  0.0  

[5 rows x 22 columns]


In [34]:
columns_to_drop = missing_data[missing_data['Percent'] > 10].index.tolist()
enhanced_data_with_no_nulls = enhanced_data.drop(columns=columns_to_drop)
enhanced_data_with_no_nulls.head()
enhanced_data.shape

(7081, 23)

## 3. Declare Data Types

Messy CSVs quietly produce object columns for booleans, numerics, and dates. Everything slows down, comparisons misfire, and merges break.

In [35]:
enhanced_data_with_no_nulls.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR'],
      dtype='object')

In [36]:
enhanced_data_with_no_nulls.dtypes

Date         object
HomeTeam     object
AwayTeam     object
FTHG        float64
FTAG        float64
FTR          object
HTHG        float64
HTAG        float64
HTR          object
Referee      object
HS          float64
AS          float64
HST         float64
AST         float64
HF          float64
AF          float64
HC          float64
AC          float64
HY          float64
AY          float64
HR          float64
AR          float64
dtype: object

In [37]:
# Convert data dtypes to actual types
enhanced_data_with_no_nulls.sample(3)


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
4638,19/01/14,Chelsea,Man United,3.0,1.0,H,2.0,0.0,H,P Dowd,...,6.0,4.0,15.0,21.0,5.0,4.0,1.0,3.0,0.0,1.0
4249,11/01/15,Man United,Southampton,0.0,1.0,A,0.0,0.0,D,P Dowd,...,0.0,1.0,9.0,10.0,3.0,5.0,2.0,3.0,0.0,0.0
3704,12/09/2015,Norwich,Bournemouth,3.0,1.0,H,1.0,0.0,H,M Atkinson,...,4.0,2.0,10.0,11.0,4.0,4.0,2.0,3.0,0.0,0.0


In [38]:
# == Fixing Date Column x Augmenting "Date" Column == #

enhanced_data_with_no_nulls['Date'] = pd.to_datetime(enhanced_data_with_no_nulls['Date'],format = 'mixed')
enhanced_data_with_no_nulls['Month'] = enhanced_data_with_no_nulls['Date'].dt.strftime('%B')
enhanced_data_with_no_nulls['Year'] = enhanced_data_with_no_nulls['Date'].dt.year
enhanced_data_with_no_nulls['Day'] = enhanced_data_with_no_nulls['Date'].dt.strftime('%A')
enhanced_data_with_no_nulls.sample(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,Month,Year,Day
6455,2008-11-22,Portsmouth,Hull,2.0,2.0,D,1.0,0.0,H,S Attwell,...,13.0,7.0,7.0,2.0,1.0,0.0,0.0,November,2008.0,Saturday
4232,2015-01-01,Liverpool,Leicester,2.0,2.0,D,2.0,0.0,H,M Jones,...,10.0,4.0,5.0,2.0,2.0,0.0,0.0,January,2015.0,Thursday
6854,2007-08-12,Newcastle,Birmingham,2.0,1.0,H,1.0,1.0,D,R Styles,...,8.0,6.0,5.0,1.0,2.0,0.0,0.0,August,2007.0,Sunday
3810,2015-12-12,Bournemouth,Man United,2.0,1.0,H,1.0,1.0,D,A Taylor,...,12.0,5.0,7.0,2.0,1.0,0.0,0.0,December,2015.0,Saturday
337,2024-03-11,Tottenham,Aston Villa,4.0,1.0,H,0.0,1.0,A,C Pawson,...,14.0,6.0,4.0,2.0,0.0,0.0,0.0,March,2024.0,Monday
1060,2022-09-16,Aston Villa,Southampton,1.0,0.0,H,1.0,0.0,H,T Harrington,...,13.0,6.0,3.0,3.0,1.0,0.0,0.0,September,2022.0,Friday
2719,2018-12-30,Southampton,Man City,1.0,3.0,A,1.0,3.0,A,P Tierney,...,10.0,3.0,8.0,2.0,3.0,1.0,0.0,December,2018.0,Sunday
1505,2021-11-28,Brentford,Everton,1.0,0.0,H,1.0,0.0,H,D England,...,11.0,3.0,3.0,4.0,3.0,0.0,0.0,November,2021.0,Sunday
4127,2014-10-26,Man United,Chelsea,1.0,1.0,D,0.0,0.0,D,P Dowd,...,14.0,4.0,7.0,3.0,5.0,0.0,1.0,October,2014.0,Sunday
1997,2021-02-19,Wolves,Leeds,1.0,0.0,H,0.0,0.0,D,D Coote,...,8.0,7.0,9.0,1.0,0.0,0.0,0.0,February,2021.0,Friday


In [39]:
#  == Fixing String Columns == #


string_cols = ['HomeTeam', 'AwayTeam', 'FTR', 'HTR', 'Referee', 'Month', 'Day'] 

# 2. Map columns to the 'str' type
str_map = ({col: str for col in string_cols})

# 3. Apply the conversion
enhanced_data_with_no_nulls = enhanced_data_with_no_nulls.astype(str_map) 
enhanced_data_with_no_nulls.dtypes

Date        datetime64[ns]
HomeTeam            object
AwayTeam            object
FTHG               float64
FTAG               float64
FTR                 object
HTHG               float64
HTAG               float64
HTR                 object
Referee             object
HS                 float64
AS                 float64
HST                float64
AST                float64
HF                 float64
AF                 float64
HC                 float64
AC                 float64
HY                 float64
AY                 float64
HR                 float64
AR                 float64
Month               object
Year               float64
Day                 object
dtype: object

In [40]:
# == Fixing Numeric Columns == #

num_cols = ['FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'HS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR','Year']


# 2. Map columns to the 'str' type
num_map = ({num: 'Int64' for num in num_cols})

# 3. Apply the conversion
enhanced_data_with_no_nulls = enhanced_data_with_no_nulls.astype(num_map) 
enhanced_data_with_no_nulls.dtypes

Date        datetime64[ns]
HomeTeam            object
AwayTeam            object
FTHG                 Int64
FTAG                 Int64
FTR                 object
HTHG                 Int64
HTAG                 Int64
HTR                 object
Referee             object
HS                   Int64
AS                 float64
HST                  Int64
AST                  Int64
HF                   Int64
AF                   Int64
HC                   Int64
AC                   Int64
HY                   Int64
AY                   Int64
HR                   Int64
AR                   Int64
Month               object
Year                 Int64
Day                 object
dtype: object

In [41]:
enhanced_data_with_no_nulls.head(6)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,Month,Year,Day
0,2025-08-15,Liverpool,Bournemouth,4,2,H,1,0,H,A Taylor,...,10,6,7,1,2,0,0,August,2025,Friday
1,2025-08-16,Aston Villa,Newcastle,0,0,D,0,0,D,C Pawson,...,11,3,6,1,1,1,0,August,2025,Saturday
2,2025-08-16,Brighton,Fulham,1,1,D,0,0,D,S Barrott,...,15,4,3,3,3,0,0,August,2025,Saturday
3,2025-08-16,Sunderland,West Ham,3,0,H,0,0,D,R Jones,...,10,5,7,0,1,0,0,August,2025,Saturday
4,2025-08-16,Tottenham,Burnley,3,0,H,1,0,H,M Oliver,...,8,6,5,0,0,0,0,August,2025,Saturday
5,2025-08-16,Wolves,Man City,0,4,A,0,2,A,J Gillett,...,7,4,6,1,2,0,0,August,2025,Saturday


In [42]:
enhanced_data_with_no_nulls.shape

(7081, 25)

In [43]:
import numpy as np
enhanced_data_with_no_nulls = enhanced_data_with_no_nulls.replace(r'^\s*$', np.nan, regex=True)
enhanced_data_with_no_nulls = enhanced_data_with_no_nulls.dropna(how='all')
enhanced_data_with_no_nulls.shape

(7081, 25)

In [44]:
# Only keep rows where 'HomeTeam' (or another core column) actually has a value
enhanced_data_with_no_nulls = enhanced_data_with_no_nulls[enhanced_data_with_no_nulls['HomeTeam'].str.strip().astype(bool)]
enhanced_data_with_no_nulls.shape

(7081, 25)

In [45]:
#enhanced_data_with_no_nulls.isnull()
null = enhanced_data_with_no_nulls.columns[enhanced_data_with_no_nulls.isnull().any()] 
null_counts  =enhanced_data_with_no_nulls.isnull().sum()
null_counts

Date        1
HomeTeam    0
AwayTeam    0
FTHG        1
FTAG        1
FTR         0
HTHG        1
HTAG        1
HTR         0
Referee     0
HS          1
AS          1
HST         1
AST         1
HF          1
AF          1
HC          1
AC          1
HY          1
AY          1
HR          1
AR          1
Month       0
Year        1
Day         0
dtype: int64

In [46]:
# ===== Find rows with ANY missing values ===== #
rows_with_nulls = enhanced_data_with_no_nulls[enhanced_data_with_no_nulls.isnull().any(axis=1)]
print(f"Number of rows with at least one missing value: {len(rows_with_nulls)}")
print(f"\nFirst 10 rows with missing values:")
rows_with_nulls.head(10)


Number of rows with at least one missing value: 1

First 10 rows with missing values:


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,Month,Year,Day
4420,NaT,nan,nan,<NA>,<NA>,nan,<NA>,<NA>,nan,nan,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,nan,<NA>,nan


In [47]:
# ===== METHOD 2: Drop rows missing in SPECIFIC columns only ===== #
# Example: Only drop if missing in critical columns
critical_cols = ['HomeTeam', 'AwayTeam', 'Date', 'FTHG', 'FTAG']
enhanced_data_clean_v2 = enhanced_data_with_no_nulls.dropna(subset=critical_cols)
print(f"Original shape: {enhanced_data_with_no_nulls.shape}")
print(f"After dropping rows missing in {critical_cols}: {enhanced_data_clean_v2.shape}")
print(f"Rows removed: {enhanced_data_with_no_nulls.shape[0] - enhanced_data_clean_v2.shape[0]}")
print(f"Original shape: {enhanced_data_clean_v2.shape}")


Original shape: (7081, 25)
After dropping rows missing in ['HomeTeam', 'AwayTeam', 'Date', 'FTHG', 'FTAG']: (7080, 25)
Rows removed: 1
Original shape: (7080, 25)


In [48]:
rows_with_nulls = enhanced_data_clean_v2[enhanced_data_clean_v2.isnull().any(axis=1)]
print(f"Number of rows with at least one missing value: {len(rows_with_nulls)}")
print(f"\nFirst 10 rows with missing values:")
rows_with_nulls.head(10)



Number of rows with at least one missing value: 0

First 10 rows with missing values:


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,Month,Year,Day


### DATABASE CONNECTION SECTION


In [49]:
import pandas as pd
from pathlib import Path

# ===== Defining the data directories ====#
PROJECT_ROOT = Path().absolute().parent.parent
DATA_DIR = PROJECT_ROOT / "datasets" 
COMMON_DATA_DIR = DATA_DIR / "common_data"
INGESTED_DIR = DATA_DIR / "ingested_data"
ingested_csv_file = INGESTED_DIR / "enhanced_dataset.csv"

CLEANED_DIR = DATA_DIR / "cleaned_ingested_data"
cleaned_ingested_csv_file = CLEANED_DIR / "cleaned_ingested_data.csv"

In [50]:
import pandas as pd
data = pd.read_csv(cleaned_ingested_csv_file)
data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,Month,Year,Day
0,2025-08-15,Liverpool,Bournemouth,4,2,H,1,0,H,A Taylor,...,10,6,7,1,2,0,0,August,2025,Friday
1,2025-08-16,Aston Villa,Newcastle,0,0,D,0,0,D,C Pawson,...,11,3,6,1,1,1,0,August,2025,Saturday
2,2025-08-16,Brighton,Fulham,1,1,D,0,0,D,S Barrott,...,15,4,3,3,3,0,0,August,2025,Saturday
3,2025-08-16,Sunderland,West Ham,3,0,H,0,0,D,R Jones,...,10,5,7,0,1,0,0,August,2025,Saturday
4,2025-08-16,Tottenham,Burnley,3,0,H,1,0,H,M Oliver,...,8,6,5,0,0,0,0,August,2025,Saturday


In [51]:
data.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'Month', 'Year', 'Day'],
      dtype='object')

In [52]:
import psycopg2
from dotenv import load_dotenv
import os
# Get the connection string from the environment variable

load_dotenv()  # Loads .env file

db_name = os.getenv('NEON_DB_NAME')
db_role = os.getenv('NEON_DB_ROLE')
db_string = os.getenv('NEON_CONNECTION_STRING')


In [53]:
try:
    with psycopg2.connect(db_string) as conn:
        print("✅ Connection established")
        
        with conn.cursor() as cur:
            # Check if table exists
            cur.execute("SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'epl_data');")
            table_exists = cur.fetchone()[0]
            
            if table_exists:
                print("✅ Table 'epl_data' already exists")
            else:
                # Create table
                cur.execute("""
                    CREATE TABLE epl_data(
                        id SERIAL PRIMARY KEY,
                        Date DATE NULL,
                        HomeTeam VARCHAR(255) NOT NULL,
                        AwayTeam VARCHAR(255) NOT NULL,
                        FTHG SMALLINT,
                        FTAG SMALLINT,
                        FTR CHAR(1),
                        HTHG SMALLINT,
                        HTAG SMALLINT,
                        HTR CHAR(1),
                        Referee VARCHAR(255),
                        HomeShots SMALLINT,
                        AwayShots SMALLINT,
                        HST SMALLINT,
                        AST SMALLINT,
                        HF SMALLINT,
                        AF SMALLINT,
                        HC SMALLINT,
                        AC SMALLINT,
                        HY SMALLINT,
                        AY SMALLINT,
                        HR SMALLINT,
                        AR SMALLINT,
                        Month VARCHAR(255),
                        Year SMALLINT,
                        Day VARCHAR(255)
                    );
                """)
                print("✅ Table created successfully in Neon")
        
        conn.commit()
        
except Exception as e:
    print("❌ Connection failed")
    print(e)

✅ Connection established
✅ Table 'epl_data' already exists


In [54]:
## Adding the data to the database

# Create a proper independent copy and reset index
data = data.copy()
data = data.reset_index(drop=True)

print(f"✅ Data prepared for insertion: {data.shape}")
print(data.head())


✅ Data prepared for insertion: (7080, 25)
         Date     HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  2025-08-15    Liverpool  Bournemouth     4     2   H     1     0   H   
1  2025-08-16  Aston Villa    Newcastle     0     0   D     0     0   D   
2  2025-08-16     Brighton       Fulham     1     1   D     0     0   D   
3  2025-08-16   Sunderland     West Ham     3     0   H     0     0   D   
4  2025-08-16    Tottenham      Burnley     3     0   H     1     0   H   

     Referee  ...  AF  HC  AC  HY  AY  HR  AR   Month  Year       Day  
0   A Taylor  ...  10   6   7   1   2   0   0  August  2025    Friday  
1   C Pawson  ...  11   3   6   1   1   1   0  August  2025  Saturday  
2  S Barrott  ...  15   4   3   3   3   0   0  August  2025  Saturday  
3    R Jones  ...  10   5   7   0   1   0   0  August  2025  Saturday  
4   M Oliver  ...   8   6   5   0   0   0   0  August  2025  Saturday  

[5 rows x 25 columns]


In [55]:
# Rename columns
data.rename(columns={
    'HS': 'HomeShots',
    'AS': 'AwayShots'
}, inplace=True)
data.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HomeShots', 'AwayShots', 'HST', 'AST', 'HF', 'AF',
       'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'Month', 'Year', 'Day'],
      dtype='object')

In [56]:

# Reorder columns to match INSERT statement
columns_order = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG','HTR','Referee', 
                 'HomeShots', 'AwayShots', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'Month', 'Year', 'Day']

# Only keep columns that exist in the dataframe
columns_order = [col for col in columns_order if col in data.columns]

data = data[columns_order]

print(f"✅ Columns ordered: {list(data.columns)}")
print(f"✅ Data shape: {data.shape}")

# Convert to tuples
data_to_insert = [tuple(row) for row in data.values]
print(f"✅ Created {len(data_to_insert)} tuples for insertion")


✅ Columns ordered: ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HomeShots', 'AwayShots', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'Month', 'Year', 'Day']
✅ Data shape: (7080, 25)
✅ Created 7080 tuples for insertion


In [57]:
# Establish the connection and insert data
conn = None
try:
    conn = psycopg2.connect(db_string)
    cur = conn.cursor()

    # Get actual columns from the dataframe
    actual_columns = list(data.columns)
    num_columns = len(actual_columns)
    
    # Create placeholders dynamically based on actual columns
    placeholders = ', '.join(['%s'] * num_columns)
    # Convert column names to lowercase to match table schema (PostgreSQL stores unquoted names as lowercase)
    columns_str = ', '.join([col.lower() for col in actual_columns])
    
    # Build INSERT statement dynamically
    insert_query = f"INSERT INTO epl_data ({columns_str}) VALUES ({placeholders})"
    
    print(f"Inserting {num_columns} columns:")
    print(f"Columns: {actual_columns}")
    print(f"Query: {insert_query[:100]}...")

    cur.executemany(insert_query, data_to_insert)
    print(f"✅ Inserted {cur.rowcount} rows into 'epl_data' table")

    # Commit the changes to the database
    conn.commit()
    print("✅ Changes committed successfully")

except Exception as e:
    print(f"❌ Error: {e}")
    if conn:
        conn.rollback()
        print("Transaction rolled back")
finally:
    if conn:
        cur.close()
        conn.close()
        print("✅ Database connection closed")


Inserting 25 columns:
Columns: ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HomeShots', 'AwayShots', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'Month', 'Year', 'Day']
Query: INSERT INTO epl_data (date, hometeam, awayteam, fthg, ftag, ftr, hthg, htag, htr, referee, homeshots...
✅ Inserted 7080 rows into 'epl_data' table
✅ Changes committed successfully
✅ Database connection closed


In [58]:


    def filter_data_by_non_betting_features(self):
        print("Data Cleaning: Step 1: Filtering feature catalog to exclude Betting_odd=True features.......")
        logging.info("Entering the Filtering out the non betting features module.....")
        
        # Load the CSV files first
        feature_catalog_df = pd.read_csv(self.ingestion_config.feature_catalog_csv_file)
        enhanced_data_df = pd.read_csv(self.ingestion_config.ingested_csv_file,low_memory=False)
        
        # Now filter
        # Had to Convert the False to avoid the boolean confusion brought about by Singleton Comparison
        is_not_betting_odd = feature_catalog_df['Betting_odd'].astype(str).str.lower().isin(['false'])
        feature_catalog_filtered = feature_catalog_df[is_not_betting_odd].copy()
        feature_names_to_keep = feature_catalog_filtered['Feature_name'].unique()
        enhanced_data = enhanced_data_df.loc[:, enhanced_data_df.columns.isin(feature_names_to_keep)].copy()


        columns_retained = enhanced_data.columns
        logging.info(f" Here are the retained columns after removing betting odds : {columns_retained}")
        logging.info(f"The new number of columns are {enhanced_data.shape[1]} and the new number of the rows are {enhanced_data.shape[0]}" )
        return enhanced_data
    

    def removing_columns_missing_significant_chunks(self,enhanced_data):
        print("Data Cleaning:Step 2: Removing columns missing significant chunks on it.......")
        try:
            total = enhanced_data.isnull().sum().sort_values(ascending=False)
            missing_percent = (enhanced_data.isnull().sum() / len(enhanced_data)) * 100
            missing_data = pd.concat([total,missing_percent],axis=1, keys=['Total','Percent'])
            logging.info(f"These are the columns that are missing{missing_data.head(20)}")
            #Deleting the columns
            columns_to_drop = missing_percent[missing_percent > 10].index
            enhanced_data_with_no_nulls = enhanced_data.drop(columns=columns_to_drop).copy()
            print(f"Dropped {len(columns_to_drop)} columns.")
            logging.info(f"Dropped {len(columns_to_drop)} columns.")
            print(f"Dropped {columns_to_drop} columns.")
            logging.info(f"Dropped {columns_to_drop} columns.")
            logging.info(f"{enhanced_data_with_no_nulls.head()}")

            #Cleaning up remaining rows
            critical_cols = ['HomeTeam', 'AwayTeam', 'Date', 'FTHG', 'FTAG']
            enhanced_data_with_no_nulls_v2 = enhanced_data_with_no_nulls.dropna(subset=critical_cols)
            logging.info(f"Original shape: {enhanced_data_with_no_nulls.shape}")
            logging.info(f"After dropping rows missing in {critical_cols}: {enhanced_data_with_no_nulls_v2.shape}")
            logging.info(f"Rows removed: {enhanced_data_with_no_nulls.shape[0] - enhanced_data_with_no_nulls_v2.shape[0]}")
            logging.info(f"New shape: {enhanced_data_with_no_nulls_v2.shape}")

            #Just checking whether they are any "Nan" rows left
            rows_with_nulls = enhanced_data_with_no_nulls_v2[enhanced_data_with_no_nulls_v2.isnull().any(axis=1)]
            logging.info(f"Number of rows with at least one missing value: {len(rows_with_nulls)}")
            logging.info(f"\nFirst 10 rows with missing values:")
            logging.info(rows_with_nulls.head(10))




        except Exception as e:
            raise CustomException(e,sys) from e
        print("Data Cleaning:Step 2: Removing columns missing significant chunks on itâœ…")
        return enhanced_data_with_no_nulls_v2
    

    def declare_dtypes_early(self,enhanced_data_with_no_nulls_v2):
        print("Data Cleaning:Step 3: Enforcing data types.......")
        try:
            logging.info("Enforcing and Augmenting Date Data types")
            enhanced_data_with_no_nulls_v2['Date'] = pd.to_datetime(enhanced_data_with_no_nulls_v2['Date'],format = 'mixed')
            enhanced_data_with_no_nulls_v2['Month'] = enhanced_data_with_no_nulls_v2['Date'].dt.strftime('%B')
            enhanced_data_with_no_nulls_v2['Year'] = enhanced_data_with_no_nulls_v2['Date'].dt.year
            enhanced_data_with_no_nulls_v2['Day'] = enhanced_data_with_no_nulls_v2['Date'].dt.strftime('%A')
            logging.info(f"Augmented date types : {enhanced_data_with_no_nulls_v2.sample(10)}")

            # == Fixing String Columns == #
            logging.info("Enforcing string columns to avoid edge cases....")
            string_cols = ['HomeTeam', 'AwayTeam', 'FTR', 'HTR', 'Referee', 'Month', 'Day'] 
            str_map = ({col: str for col in string_cols})
            enhanced_data_with_no_nulls_v2 = enhanced_data_with_no_nulls_v2.astype(str_map) 
            logging.info(f"Here are the data types: {enhanced_data_with_no_nulls_v2.dtypes}")

            # == Fixing Numeric Columns == #
            logging.info("Enforcing numeric columns......")
            num_cols = ['FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'HS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR','Year']
            num_map = ({num: 'Int64' for num in num_cols})
            enhanced_data_with_no_nulls_v2 = enhanced_data_with_no_nulls_v2.astype(num_map)
            logging.info(f"Here are the data types: {enhanced_data_with_no_nulls_v2.dtypes}")
        except Exception as e:
            raise CustomException(e,sys) from e
        print("Data Cleaning:Step 3: Enforcing data types.......âœ…")
        logging.info("Data Cleaning:Step 3: Enforcing data types.......DONEâœ…")
        return enhanced_data_with_no_nulls_v2
    